In [2]:
import os
import sys
import cv2
import  torch
import  numpy as np
import torch.utils.data as data
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
import pandas as pd
import random
import pickle
from tqdm import tqdm
import csv

In [3]:
class Imgdataset(data.Dataset):
    def __init__(self, img_folder, ann_data, img_size=64, transform=[]):
        self.img_folder = img_folder
        self.ann_data = ann_data
        self.transform = transform

        self.all_ann_smiling = {}
        with open(ann_data, 'r') as f:
            rows = csv.DictReader(f)
            for row in rows:
                self.all_ann_smiling[row['image_name']] = int(float(row['Smiling']))
        
        self.all_img_path = []
        all_img_file = os.listdir(img_folder)
        for file_name in all_img_file:
            self.all_img_path.append(file_name)

        
    def __len__(self):
        return len(self.all_img_path)   

    def __getitem__(self, index):
        
        img_file_name = self.all_img_path[index]
        label = self.all_ann_smiling[img_file_name]
        
        img_path = os.path.join(self.img_folder, img_file_name)
        img = cv2.imread(img_path)
        img = self.BGR2RGB(img)
        #img = self.random_flip(img)
        for t in self.transform:
            img = t(img)

        return img, label


    def BGR2RGB(self,img):
        return cv2.cvtColor(img,cv2.COLOR_BGR2RGB)

In [4]:
class Generator(nn.Module):
    def __init__(self, figsize=64):
        super(Generator, self).__init__()
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d( 101, figsize * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(figsize * 8),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(figsize * 8, figsize * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(figsize * 4),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(figsize * 4, figsize * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(figsize * 2),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(figsize * 2, figsize, 4, 2, 1, bias=False),
            nn.BatchNorm2d(figsize),
            nn.ReLU(inplace=True),
            nn.ConvTranspose2d(figsize, 3, 4, 2, 1, bias=False),
            nn.Tanh()
        )
            
    def forward(self, X):
        output = self.decoder(X)/2.0+0.5
        return output
    
class Discriminator(nn.Module):
    def __init__(self, figsize=64):
        super(Discriminator, self).__init__()
        self.decoder = nn.Sequential(
            nn.Conv2d(3, figsize, 4, 2, 1),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(figsize, figsize * 2, 4, 2, 1),
            nn.BatchNorm2d(figsize * 2),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(figsize * 2, figsize * 4, 4, 2, 1),
            nn.BatchNorm2d(figsize * 4),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(figsize * 4, figsize * 8, 4, 2, 1),
            nn.BatchNorm2d(figsize * 8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(figsize * 8, figsize *1, 4, 1, 0),
        )
        self.fc_dis = nn.Linear(figsize *1, 1)
        self.fc_aux = nn.Linear(figsize *1, 1) 
       
        self.softmax = nn.Softmax()
        self.sigmoid = nn.Sigmoid()

    def forward(self, X):
        decode_output = self.decoder(X)

        flat = decode_output.view(-1,64)
        fc_dis = self.fc_dis(flat)
        fc_aux = self.fc_aux(flat)
        
        realfake = self.sigmoid(fc_dis)
        classes = self.sigmoid(fc_aux)
        
        return realfake, classes

In [5]:
random.seed((38))
torch.manual_seed(38)
up = np.ones(10)
down = np.zeros(10)
fixed_class = np.hstack((up,down))
fixed_class = torch.from_numpy(fixed_class).view(20,1,1,1).type(torch.FloatTensor)
fixed_noise = torch.randn(10, 100, 1, 1)
fixed_noise = torch.cat((fixed_noise,fixed_noise))
fixed_input = Variable(torch.cat((fixed_noise, fixed_class),1)).cuda()

In [15]:
train_file_img = "../../../hw3_data/face/train"
csv_data_path = "../../../hw3_data/face/train.csv"
img_size = 64
train_data = Imgdataset(train_file_img, csv_data_path, img_size, transform=[transforms.ToTensor()])

dataloader = torch.utils.data.DataLoader(
                                dataset=train_data,
                                batch_size=64,
                                shuffle=True,
                                num_workers=8
                            )

In [35]:
# loss functions
dis_criterion = nn.BCELoss()
aux_criterion = nn.BCELoss()

# training
latent_size = 100
BATCH_SIZE = 64
G = Generator()
D = Discriminator()
G.cuda()
D.cuda()

# setup optimizer
beta_1 = 0.5
optimizerG = optim.Adam(G.parameters(), lr=0.0002, betas=(beta_1,0.999))
optimizerD = optim.Adam(D.parameters(), lr=0.0002, betas=(beta_1,0.999))


D_loss_list = []
G_loss_list = []
D_fake_acc_list = []
D_real_acc_list = []

D_fake_class_list = []
D_real_class_list = []


for epoch in range(200):
    print("Epoch:", epoch+1)
    epoch_D_loss = 0.0
    epoch_G_loss = 0.0
    D_fake_acc = 0.0
    D_real_acc = 0.0
    D_fake_class = 0.0
    D_real_class = 0.0
    total_length = len(dataloader)

    if (epoch+1) == 15:
        optimizerG.param_groups[0]['lr'] /= 2
        optimizerD.param_groups[0]['lr'] /= 2
        print("learning rate change!")
    
    
    for i, batch in enumerate(dataloader):
        if i > total_length:
            break
        for _ in range(1):    
            D.zero_grad()
            input_X = batch[0]
            intput_class = batch[1]

            #### train with real image -> ground truth = real label
            real_image = Variable(input_X.cuda()) # use GPU 
            real_class = Variable(intput_class.type(torch.FloatTensor).cuda())
            real_label = Variable(torch.ones((BATCH_SIZE))).cuda()
            dis_ouput, aux_output = D(real_image)
            D_real_dis_loss = dis_criterion(dis_ouput, real_label.view(BATCH_SIZE,1))
            D_real_aux_loss = aux_criterion(aux_output, real_class.view(BATCH_SIZE,1))
            D_real_acc += np.mean(((dis_ouput > 0.5).cpu().data.numpy() == real_label.cpu().data.numpy()))
            D_real_loss = (D_real_dis_loss + D_real_aux_loss)/2
            D_real_class += D_real_aux_loss.item()

            noise = torch.randn(BATCH_SIZE, 100, 1, 1)
            fake_class = torch.from_numpy(np.random.randint(2,size=BATCH_SIZE)).view(BATCH_SIZE,1,1,1)
            intput_vector =Variable(torch.cat((noise,fake_class.type(torch.FloatTensor)),1)).cuda()

            fake_label = Variable(torch.zeros((BATCH_SIZE))).cuda()
            fake_class = Variable(fake_class.type(torch.FloatTensor)).cuda()

            fake_image = G(intput_vector)
            dis_output, aux_output = D(fake_image.detach())
            D_fake_dis_loss = dis_criterion(dis_output, fake_label.view(BATCH_SIZE,1))
            D_fake_aux_loss = aux_criterion(aux_output, fake_class.view(BATCH_SIZE,1))
            D_fake_loss = (D_fake_dis_loss + D_fake_aux_loss)/2
            D_fake_acc += np.mean(((dis_output > 0.5).cpu().data.numpy() == fake_label.cpu().data.numpy()))
            D_fake_class += D_fake_aux_loss.item()
            # update D
            D_train_loss = D_real_loss + D_fake_loss
            D_train_loss.backward()
            epoch_D_loss+=(D_train_loss.item())

            optimizerD.step()
        
        #### train Generator
        for _ in range(2):
            G.zero_grad()

            noise = torch.randn(BATCH_SIZE, 100, 1, 1)
            fake_class = torch.from_numpy(np.random.randint(2,size=BATCH_SIZE)).view(BATCH_SIZE,1,1,1)

            intput_vector = Variable(torch.cat((noise,fake_class.type(torch.FloatTensor)),1)).cuda()

            fake_class = Variable(fake_class.type(torch.FloatTensor)).cuda()
            fake_label_for_G = Variable(torch.ones((BATCH_SIZE))).cuda()

            fake_image = G(intput_vector)
            dis_output, aux_output = D(fake_image)
            G_dis_loss = dis_criterion(dis_output, fake_label_for_G.view(BATCH_SIZE,1))
            G_aux_loss = aux_criterion(aux_output, fake_class.view(BATCH_SIZE,1))
            G_train_loss = G_dis_loss + G_aux_loss
            G_train_loss.backward()
            optimizerG.step()
        epoch_G_loss += (G_train_loss.item())
    print("training D Loss:",epoch_D_loss/(total_length))
    print("training G Loss:", epoch_G_loss/(total_length))
    D_loss_list.append(epoch_D_loss/(total_length))
    G_loss_list.append(epoch_G_loss/(total_length))
    
    print("D_real_dis_acc:", D_real_acc/(total_length/BATCH_SIZE))
    print("D_fake_dis_acc:", D_fake_acc/(total_length/BATCH_SIZE))
    print("D_real_aux_loss:", D_real_class/(total_length/BATCH_SIZE))
    print("D_fake_aux_loss:", D_fake_class/(total_length/BATCH_SIZE))
    D_real_acc_list.append(D_real_acc/(total_length/BATCH_SIZE))
    D_fake_acc_list.append(D_fake_acc/(total_length/BATCH_SIZE))    
    D_real_class_list.append(D_real_class/(total_length/BATCH_SIZE))
    D_fake_class_list.append(D_fake_class/(total_length/BATCH_SIZE))
    # evaluation
    G.eval()
    fixed_img_output = G(fixed_input)
    G.train()
    torchvision.utils.save_image(fixed_img_output.cpu().data, './output3/output_'+str(epoch+1)+'.jpg',nrow=10)
    
    torch.save(G.state_dict(), "./checkpoint3/G_ACG2_model.pkt."+str(epoch+1))
    torch.save(D.state_dict(), "./checkpoint3/D_ACG2_model.pkt."+str(epoch+1))

Epoch: 1
training D Loss: 0.9111850221633911
training G Loss: 1.181247861289978
D_real_dis_acc: 34.5728
D_fake_dis_acc: 35.4704
D_real_aux_loss: 22.557610455322266
D_fake_aux_loss: 4.1843146078109745
Epoch: 2
training D Loss: 0.823833782863617
training G Loss: 0.9617933699607849
D_real_dis_acc: 35.4304
D_fake_dis_acc: 37.7264
D_real_aux_loss: 17.300579410552977
D_fake_aux_loss: 1.1598817717552186
Epoch: 3
training D Loss: 0.8006055232048035
training G Loss: 0.9560264995574951
D_real_dis_acc: 36.6624
D_fake_dis_acc: 38.1808
D_real_aux_loss: 15.276254290771485
D_fake_aux_loss: 1.1510674974441528
Epoch: 4
training D Loss: 0.7910450421333313
training G Loss: 0.9165473872184754
D_real_dis_acc: 36.7824
D_fake_dis_acc: 37.9552
D_real_aux_loss: 14.204255059051514
D_fake_aux_loss: 1.243900582385063
Epoch: 5
training D Loss: 0.7742502745628357
training G Loss: 0.948410243177414
D_real_dis_acc: 38.1536
D_fake_dis_acc: 38.8032
D_real_aux_loss: 13.27527667617798
D_fake_aux_loss: 1.289431622505188
E

training D Loss: 0.15181630113720893
training G Loss: 4.9179452699661255
D_real_dis_acc: 61.0512
D_fake_dis_acc: 60.8368
D_real_aux_loss: 1.0575666549295186
D_fake_aux_loss: 1.2354019808603451
Epoch: 43
training D Loss: 0.19215094296336174
training G Loss: 4.95041768527031
D_real_dis_acc: 60.0352
D_fake_dis_acc: 59.7808
D_real_aux_loss: 1.182758560037613
D_fake_aux_loss: 1.3970985364573076
Epoch: 44
training D Loss: 0.15954525125622748
training G Loss: 4.998869431304931
D_real_dis_acc: 60.7664
D_fake_dis_acc: 60.5488
D_real_aux_loss: 1.0669844166755675
D_fake_aux_loss: 1.2383738982167094
Epoch: 45
training D Loss: 0.18154515961110593
training G Loss: 5.023114916038513
D_real_dis_acc: 60.16
D_fake_dis_acc: 59.7568
D_real_aux_loss: 1.1594472771823405
D_fake_aux_loss: 1.3741188625900076
Epoch: 46
training D Loss: 0.17877074747383595
training G Loss: 5.123829780054092
D_real_dis_acc: 60.2528
D_fake_dis_acc: 60.0912
D_real_aux_loss: 1.2472111391991376
D_fake_aux_loss: 1.4755810944546015
Epo

training D Loss: 0.14189015330523252
training G Loss: 6.258035359287262
D_real_dis_acc: 61.3712
D_fake_dis_acc: 61.1456
D_real_aux_loss: 0.9186446513146163
D_fake_aux_loss: 1.1990005317328498
Epoch: 84
training D Loss: 0.1286069469563663
training G Loss: 6.293020923137664
D_real_dis_acc: 61.696
D_fake_dis_acc: 61.4
D_real_aux_loss: 0.9256240832686424
D_fake_aux_loss: 1.2994342371090315
Epoch: 85
training D Loss: 0.13120153655856848
training G Loss: 6.1836177277565
D_real_dis_acc: 61.528
D_fake_dis_acc: 61.2848
D_real_aux_loss: 0.8161172852903604
D_fake_aux_loss: 1.1697780073984525
Epoch: 86
training D Loss: 0.13472361371219158
training G Loss: 6.184031037759781
D_real_dis_acc: 61.4656
D_fake_dis_acc: 61.2752
D_real_aux_loss: 0.7854646176189184
D_fake_aux_loss: 1.3329733927787282
Epoch: 87
training D Loss: 0.12254385867863893
training G Loss: 6.294933813476563
D_real_dis_acc: 61.7424
D_fake_dis_acc: 61.5008
D_real_aux_loss: 0.8164220487624407
D_fake_aux_loss: 1.2575173270476983
Epoch: 8

training D Loss: 0.09589251101911068
training G Loss: 7.0783509604454045
D_real_dis_acc: 62.3776
D_fake_dis_acc: 62.2448
D_real_aux_loss: 0.7109400512427092
D_fake_aux_loss: 1.3908422490950674
Epoch: 125
training D Loss: 0.11469513545408845
training G Loss: 7.223625423622131
D_real_dis_acc: 61.9168
D_fake_dis_acc: 61.688
D_real_aux_loss: 0.7104076077222824
D_fake_aux_loss: 0.9689459162730258
Epoch: 126
training D Loss: 0.09770737611949444
training G Loss: 7.1615738733291625
D_real_dis_acc: 62.1776
D_fake_dis_acc: 62.0432
D_real_aux_loss: 0.6783396767899394
D_fake_aux_loss: 0.9724351798757911
Epoch: 127
training D Loss: 0.08387158966735006
training G Loss: 7.132795063972473
D_real_dis_acc: 62.5088
D_fake_dis_acc: 62.408
D_real_aux_loss: 0.6319580152601003
D_fake_aux_loss: 1.016373097742442
Epoch: 128
training D Loss: 0.09061391914263367
training G Loss: 7.242408863067627
D_real_dis_acc: 62.4864
D_fake_dis_acc: 62.3568
D_real_aux_loss: 0.6678793776020407
D_fake_aux_loss: 1.29172450312571

training D Loss: 0.09883706137686968
training G Loss: 7.559238776779175
D_real_dis_acc: 62.1552
D_fake_dis_acc: 62.0496
D_real_aux_loss: 0.57547197599262
D_fake_aux_loss: 1.2344295302983372
Epoch: 166
training D Loss: 0.10891657056808472
training G Loss: 7.549034243965149
D_real_dis_acc: 62.0688
D_fake_dis_acc: 61.92
D_real_aux_loss: 0.6079664593085646
D_fake_aux_loss: 1.4108734742912463
Epoch: 167
training D Loss: 0.08446296211108566
training G Loss: 7.630634232330323
D_real_dis_acc: 62.5504
D_fake_dis_acc: 62.3408
D_real_aux_loss: 0.598999728512764
D_fake_aux_loss: 1.2482963443958202
Epoch: 168
training D Loss: 0.09348692250698805
training G Loss: 7.614599384880066
D_real_dis_acc: 62.408
D_fake_dis_acc: 62.2432
D_real_aux_loss: 0.6586786460161209
D_fake_aux_loss: 1.5932017828548326
Epoch: 169
training D Loss: 0.11661626283153892
training G Loss: 7.733311151409149
D_real_dis_acc: 61.8864
D_fake_dis_acc: 61.7072
D_real_aux_loss: 0.6807973698824644
D_fake_aux_loss: 1.2820731237642466
Ep

In [17]:
### Load Model ###
AC_G = Generator()
AC_G.cuda()
pretrained_acgan = "./checkpoint3/G_ACG2_model.pkt.196"
AC_G.load_state_dict(torch.load(pretrained_acgan))
AC_G.eval()

output1 = []
output2 = []
random_seed = [1046, 33,45,50,60,84,169,120,1065,9093]
for seed in random_seed[:]:
    random.seed((seed))
    torch.manual_seed(seed)
    # use for random generation
    up = np.ones(1)
    down = np.zeros(1)
    fixed_class = np.hstack((up,down))
    fixed_class = torch.from_numpy(fixed_class).view(2,1,1,1).type(torch.FloatTensor)
    fixed_noise = torch.randn(1, 100, 1, 1)
    fixed_noise = torch.cat((fixed_noise,fixed_noise))
    fixed_input = Variable(torch.cat((fixed_noise, fixed_class),1)).cuda()

    fixed_img_output = AC_G(fixed_input)
    
    output1.append(fixed_img_output[0].data.unsqueeze(0))
    output2.append(fixed_img_output[1].data.unsqueeze(0))

output_pair_1 = torch.cat(output1)
output_pair_2 = torch.cat(output2)
print(output_pair_1.size())
print(output_pair_2.size())

output = torch.cat((output_pair_1, output_pair_2))
    
torchvision.utils.save_image(output, "./seed_2/seed_"+"ALL"+".jpg",nrow=10)

torch.Size([10, 3, 64, 64])
torch.Size([10, 3, 64, 64])
